In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import GEOparse
import numpy as np

#### Local version (testing)

In [2]:
if False:
    connection = pymysql.connect(host='localhost',
                                 user='root',
                                 #password='12345',
                                 db='parent')
    cursor = connection.cursor()

#### InSyBio server version (application)

In [16]:
connection = pymysql.connect(host='server.com',
                             user='user_name',
                             password='213454',
                             db='db_name')
cursor = connection.cursor()

In [29]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

(('information_schema',),
 ('parent',),
 ('parent_etaireia',),
 ('parent_univdb',),
 ('performance_schema',))

In [30]:
cursor.execute("SHOW DATABASES") # Show the available databases in the host
print("The available databases are:")
DB_names = []
for x in cursor:
    #print(*x)
    DB_names.append(*x)
print(DB_names,end="\n\n")

cursor.execute("SHOW TABLES") # Show the tables of a specific database
print("The available tables are:")
table_names = []
for x in cursor:
    #print(x)
    table_names.append(*x)
print(table_names)

The available databases are:
['information_schema', 'parent', 'parent_etaireia', 'parent_univdb', 'performance_schema']

The available tables are:
['dataset_to_differentialexpression', 'dataset_to_phenotypes', 'dataset_to_tissues', 'datasets', 'differential_expression', 'gene_to_transcript', 'genes', 'medication', 'phenotypes', 'preprocessed_data', 'processed_data', 'raw_dataset', 'sample_to_medication', 'sample_to_treatment', 'samples', 'tissues', 'transcript_expression', 'transcripts', 'treatment']


In [5]:
if connection.db:
    #print(str(connection.db).split("\'")[1])
    db = str(connection.db).split("\'")[1]

cursor.execute("SHOW TABLES")
print(f'The tables in the {db} DataBase are:\n')
for i in cursor.fetchall():
    print(i[0])

The tables in the parent DataBase are:

dataset_to_differentialexpression
dataset_to_phenotypes
dataset_to_tissues
datasets
differential_expression
gene_to_transcript
genes
medication
phenotypes
preprocessed_data
processed_data
raw_dataset
sample_to_medication
sample_to_treatment
samples
tissues
transcript_expression
transcripts
treatment


# Functions

In [4]:
def add_row(engine, cursor, table, columns, values, verbose = False):
    ''' Adds an instance in a table as a row.
    Inputs: 
    <object> engine. The connection to the database
    <Object> curasor. The entity that \'talks\' to the database
    <str> table. The table's name we need to add the instance
    <list-like> values. The values we need to add to the table of the database
    '''
    columns = tuple(columns)
    # if str(type(df)).startswith("<class \'pandas.core"):
    #     values = list(values.itertuples(index=False))
    print(f'This is the original: {values}')
    values = tuple(values)
    positions = len(columns)
    sql = f"INSERT INTO {table} ({','.join([str(s) for s in list(columns)])}) VALUES ({str('%s, '*len(columns)).strip().strip(',')})" #SQL command structure
    if verbose:
        print(values)
        print(type(values))
    cursor.executemany(sql, [values]) # Execute multiple lines
    engine.commit() # Commit the change to happen

    print(cursor.rowcount, "record inserted.")
    print(f'Last ID: {cursor.lastrowid}')

In [5]:
def restore_ID(table):
    '''Restores the Ids of a table in the database
    Input: <str> table. The table's name we need to restore the ID'''
    try:
        sql = f"ALTER TABLE {table} AUTO_INCREMENT = 1" #SQL command structure
        cursor.execute(sql)
        connection.commit()
        print('Prmary Key restored!')
    except:
        print('Prmary Key restoration failed!')
        print('call the \'restore_ID(table_name)\' again with the correct name!')

In [6]:
def delete_all_in_table(table):
    if str(input(f'This will delete all instances of the {table} table.\nContinue? [Y/N]: ')).lower() in ['y','yes']:
        try:
            sql = f"DELETE FROM {table}" #SQL command structure
            cursor.execute(sql)
            connection.commit()
            print(f'All instances of the {table} table deleted. The table is now empty.')
            if str(input(f'Restore the Primary Key ID? [Y/N]: ')).lower() in ['y','yes']:
                restore_ID(table)
        except:
            print(f'Check the name of the table. It seems that {table} is no in the database.')
    else:
        print(f'Action aborted. No cange made in {table} table')

In [7]:
def make_engine(host='localhost', user='root', passord='', db='parent'):
    engine = create_engine(f"mysql+pymysql://{user}:{passord}@{host}/{db}")
    print(f"mysql+pymysql://{user}:{passord}@{host}/{db}")
    return engine

In [8]:
def drop_column(table, columns):
    columns = tuple(columns)
    table = 'datasets'
    flat_str = ', '.join([f"DROP COLUMN {i}" for i in columns])
    sql_query = f'ALTER TABLE datasets {flat_str}'
    cursor.execute(sql_query)
    

In [9]:
def drop_PrimaryKey(table, key):
    sql = f"ALTER TABLE {table} DROP INDEX {key}; "
    cursor.execute(sql)
    connection.commit()
    

In [10]:
def index_subset_data(index, dataframe):
    print(dataframe.iloc[index,],sep='\n\n')
    return dataframe.iloc[index,]

In [11]:
def table_subset_data(table, dataframe):
    #dataframe_tmp = dataframe.copy()
    #dataframe_tmp.set_index(dataframe_tmp.columns.values[0], inplace=True)
    table = str(table).lower()
    if table=='datasets':
        index = ['GEO_ID', 'Dataset_url', 'Platform_ID', 'Platform_name', 'Expererimental_protocol']
    
    elif table=='raw_dataset':
        index = ['Raw_path']
    
    elif table=='preprocessed_data':
        index = ['Preprocessed_url', 'Preprocessed_download', 'Preprocessing_description']
    
    elif table=='processed_data':
        index = ['Processed_path', 'Processed_path_integration']
    
    elif table=='differential_expression':
        index = ['Source_file_path', 'Volcano_file_path', 'boxplot_file_path', 'heatmap_file_path']
    
    elif table=='phenotypes':
        index = ['Phenotype_name']
    
    elif table=='tissues':
        index = ['tissue_name']
    
    #index = list(filter(lambda x: dataframe.index[x] in index, range(len(dataframe))))
    print(dataframe.loc[index,],sep='\n\n')
    return dataframe.loc[index,]

In [12]:
def get_auto_ID(table):
    cursor.execute(f'SELECT `AUTO_INCREMENT` FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = %s',args=table)
    current_ID = cursor.fetchone()[0] - 1
    next_ID = current_ID + 1
    print(f'Current ID = {current_ID}\nNext ID = {next_ID}')
    return current_ID, next_ID

# PyMySQL

In [11]:
# GSE6575
gse = input('Provide the GEO accession number for the Series\nLeave empty for the example dataset GSE6575: ')
if len(gse)==0:
    gse = 'GSE6575'
data = {}
if gse.startswith('GSE'):
    print('Initiating the parshing of data from GEO')
    data[gse] = GEOparse.get_GEO(geo=gse)#, silent=True)
else:
    print(f'{gse} seems not to be a GEO Series, because it doesn\'t start with \'GSE...\'')

Provide the GEO accession number for the Series
Leave empty for the example dataset GSE6575:  


31-Jul-2023 11:50:45 DEBUG utils - Directory ./ already exists. Skipping.
31-Jul-2023 11:50:45 INFO GEOparse - File already exist: using local version.
31-Jul-2023 11:50:45 INFO GEOparse - Parsing ./GSE6575_family.soft.gz: 
31-Jul-2023 11:50:45 DEBUG GEOparse - DATABASE: GeoMiame
31-Jul-2023 11:50:45 DEBUG GEOparse - SERIES: GSE6575
31-Jul-2023 11:50:45 DEBUG GEOparse - PLATFORM: GPL570


Initiating the parshing of data from GEO


C:\ProgramData\Anaconda3\lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
31-Jul-2023 11:50:46 DEBUG GEOparse - SAMPLE: GSM151962
31-Jul-2023 11:50:46 DEBUG GEOparse - SAMPLE: GSM151963
31-Jul-2023 11:50:46 DEBUG GEOparse - SAMPLE: GSM151964
31-Jul-2023 11:50:46 DEBUG GEOparse - SAMPLE: GSM151965
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151966
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151967
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151968
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151969
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151970
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151971
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151972
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151973
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPLE: GSM151974
31-Jul-2023 11:50:47 DEBUG GEOparse - SAMPL

In [266]:
cursor.execute(f'SELECT `AUTO_INCREMENT` FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = %s',args=table)
cursor.fetchall()

((3,),)

True

In [ ]:
data[list(data.keys())[0]]

# Read Excel data file

In [115]:
import os
import glob
import shutil

#filename = os.path.join(dirname, 'relative/path/to/file/you/want')
starting_path =  os.getcwd()
print(os.getcwd())
files = glob.glob('*.csv')

files

if False:
    os.chdir("./Bioinfo_WG/")
    print(os.getcwd())

b = glob.glob('GSE*')
b = [f for f in b if not '.' in f]


for dest in b:
    print(dest)
    shutil.copyfile(files[0], dst=os.path.join(os.getcwd(),dest,f'{dest}_info.csv'))

C:\Users\UTENTE\Desktop\PoliTo\PARENT\07-Projects\Bioinformatics_WG\Bioinfo_WG
GSE111175


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\UTENTE\\Desktop\\PoliTo\\PARENT\\07-Projects\\Bioinformatics_WG\\Bioinfo_WG\\GSE111175\\GSE111175_info.csv'

In [101]:
gse = b[0]
print(gse)

GSE111175


In [13]:
data_file = pd.read_csv(f'C:/Users/UTENTE/Desktop/PoliTo/PARENT/07-Projects/Bioinformatics_WG/Bioinfo_WG/{gse}/{gse}_info.csv', header=None, index_col=0)

In [285]:
data_file

,1
0,
Datasets_ID,NaN
GEO_ID,GSE6575
Dataset_url,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
Platform_ID,GPL570
Platform_name,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...
Expererimental_protocol,Gene expression in blood of children with auti...
Raw_path,C:\Users\UTENTE\Desktop\PoliTo\PARENT\07 - Pro...
Preprocessed_url,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
Preprocessed_download,https://www.ncbi.nlm.nih.gov/geo/download/?acc...


In [320]:
cursor.execute(f'SELECT GEO_ID FROM datasets')
ids = cursor.fetchall()
geo_ID = data_file.loc['GEO_ID'].values[0]
if geo_ID in [ls[0] for ls in ids]:
    print(f'This GEO ID ({geo_ID}) has been found in the database.')
    cursor.execute('SELECT Dataset_ID FROM datasets WHERE GEO_ID = %s', geo_ID)
    print(cursor.fetchall())
else:
    print(f'Starting the insertion of the dataset with GEO ID: {geo_ID}.')
    cursor.execute('SELECT Dataset_ID FROM datasets WHERE GEO_ID = %s', geo_ID)
    cursor.fetchone()

This GEO ID (GSE6575) has been found in the database.
((1,), (2,))


In [314]:
geo_ID

array(['GSE6575'], dtype=object)

In [318]:
cursor.execute('SELECT * FROM datasets WHERE GEO_ID = %s', [geo_ID[0]])

2

In [308]:
cursor.execute('SELECT * FROM datasets WHERE Dataset_ID = %s', [2])
cursor.fetchone()

(2,
 'GSE6575',
 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE6575',
 'GPL570',
 '[HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array',
 'Gene expression in blood of children with autism spectrum disorder (ASD) was studied. Transcriptional profiles were compared with age and gender matched, typically developing children from the general population (GP) or IQ matched children with mental retardation or developmental delay (MR/DD).')

In [292]:
current_id, upcomming_id = get_auto_ID('datasets')

Current ID = 2
Next ID = 3


In [298]:
cursor.execute('SELECT LAST_INSERT_ID()')
cursor.fetchone()

(2,)

In [221]:
# dataset_table = index_subset_data(range(0,4), data_file)
# RawData_table = index_subset_data(range(4,5), data_file)
# PreprocessedData_table = index_subset_data(range(5,8), data_file)
# processedData_table = index_subset_data(range(8,10), data_file)
# processedData_table = index_subset_data(range(10,11), data_file)
# DiffExp_table = index_subset_data(range(10,14), data_file)
# Pheno_table = index_subset_data(range(14,15), data_file)
# Tissue_table = index_subset_data(range(15,16), data_file)

table_name = 'Datasets'
dataset_table = table_subset_data(table_name, data_file)
df = dataset_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'raw_dataset'
RawData_table = table_subset_data(table_name, data_file)
df = RawData_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'preprocessed_data'
PreprocessedData_table = table_subset_data(table_name, data_file)
df = PreprocessedData_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'processed_data'
processedData_table = table_subset_data(table_name, data_file)
df = processedData_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'differential_expression'
DiffExp_table = table_subset_data(table_name, data_file)
df = DiffExp_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'phenotypes'
Pheno_table = table_subset_data(table_name, data_file)
df = Pheno_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])

table_name = 'tissues'
Tissue_table = table_subset_data(table_name, data_file)
df = Tissue_table.copy()
add_row(connection, cursor, table_name, df.index.values, df.values[:,0])


                                                                         1
0                                                                         
GEO_ID                                                             GSE6575
Dataset_url              https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
Platform_ID                                                         GPL570
Platform_name            [HG-U133_Plus_2] Affymetrix Human Genome U133 ...
Expererimental_protocol  Gene expression in blood of children with auti...
This is the original: ['GSE6575' 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE6575'
 'GPL570' '[HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array'
 'Gene expression in blood of children with autism spectrum disorder (ASD) was studied. Transcriptional profiles were compared with age and gender matched, typically developing children from the general population (GP) or IQ matched children with mental retardation or developmental delay (MR/DD).']
('GSE657

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`parent`.`raw_dataset`, CONSTRAINT `raw_to_dataset_fk` FOREIGN KEY (`Datasets_ID`) REFERENCES `datasets` (`Dataset_ID`) ON DELETE NO ACTION ON UPDATE NO ACTION)')

# Insertion of data to the DB

## Get information for all Series

In [ ]:
data.phenotype_data.head()

In [ ]:
data.phenotype_data.columns

In [ ]:
data.metadata.keys()

In [ ]:
data.metadata['summary']

In [ ]:
data.gsms[list(data.gsms.keys())[0]].columns

## Get the Data of 1 sample

In [ ]:
data.gsms[list(data.gsms)[0]].table # Transcript_expression table linked to the datasets

## Get GPL (Platform) info. Like f <- fData(gse) in R

In [ ]:
if len(list(data.gpls))>1: 
    print('There are {len(list(data.gpls))} GPLs')
data.gpls[list(data.gpls)[0]].table

### Get information about the GPL columns

In [ ]:
data.gpls[list(data.gpls.keys())[0]].columns

### Create Variables starting with _DB_ with the names of all the tables of the database

In [ ]:
column_names = {}
for nome in table_names:
    cursor.execute(f'SHOW COLUMNS FROM {nome}')
    column_names[nome] = []
    for x in cursor:
        #print(x)
        column_names[nome].append(list(x)[0])
    #print('\n',column_names)
    #globals()[f"_DB_{nome}"] = pd.DataFrame(column_names)
Database = pd.Series(column_names[list(column_names.keys())[0]], name=list(column_names.keys())[0])
for i in range(1,len(column_names.keys())):
    Database = pd.concat([Database, pd.Series(column_names[list(column_names.keys())[i]], name=list(column_names.keys())[i])], axis=1)


In [ ]:
#cursor.execute('ALTER TABLE `datasets` ADD `Description` TEXT CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL ; ')

In [ ]:
cursor.execute('SHOW COLUMNS FROM datasets')

column_names = []
columns = []
for x in cursor:
    print(x)
    columns.append(list(x))
    column_names.append(list(x)[0])
print('\n',column_names)

In [ ]:
print(np.setdiff1d(Database.datasets.values.flatten(), insert_data.columns))

columns = pd.DataFrame(np.array(columns)[:,1:], index=np.array(columns)[:,0], columns=['Type','Null','Key_type','Default','Auto_fill_type'])

columns.loc[np.setdiff1d(Database.datasets.values.flatten(), insert_data.columns),'Null']

In [ ]:
for col_name in columns.index:
    
    if col_name in insert_data.columns:
        #print(col_name, 'Value: ', insert_data[col_name].values[0])
        columns.loc[col_name,'value'] = insert_data[col_name].values[0]
    else: columns.loc[col_name,'value'] = ''
    
    #if columns.loc[col_name,'Null']=='NO' and columns.loc[col_name,'value'] == '': 
        #print(f'The column {col_name} needs to be filled')
        #print('These columns will take a value of 9999')
columns.loc[(columns['Null']=='NO') & (columns['Auto_fill_type']!='auto_increment') & (columns['value']==''), 'value'] = 9999

In [ ]:
columns

In [ ]:
table = 'datasets'
values = tuple(columns['value'])

In [ ]:
#columns.T.iloc[-1,:].to_sql(table, con=connection, if_exists='append', index=False)

In [ ]:
cursor.execute('SHOW ENGINE INNODB STATUS')
query = 'LATEST FOREIGN KEY ERROR'
for i in cursor:
    print(i)

In [ ]:
restore_ID(table='datasets')

In [ ]:
add_row(connection, cursor,table, columns=tuple(columns.index.values), values=values)

In [ ]:
delete_all_in_table('datasets')

In [ ]:
if False:
    sql = "ALTER TABLE datasets DROP INDEX foreign_key_12; " #Else run the function drop_PrimaryKey(table,key)
    cursor.execute(sql)
    connection.commit()

In [ ]:
if False:
    drop_column('dataset',('Raw_ID','Processed_ID'))

In [ ]:
data.metadata

In [ ]:
patien_1 = GEOparse.(data.metadata['sample_id'][0])

In [ ]:
data.metadata['sample_id'][0]

In [ ]:
gpl = GEOparse.GPL(data.metadata['platform_id'][0])

In [ ]:
data.gpls[list(data.gpls)[0]].table

In [ ]:
data.gsms[list(data.gsms.keys())[5]].annotate(data.gpls[list(data.gpls)[0]].table, annotation_column='ENTREZ_GENE_ID')

In [ ]:
data.gpls[list(data.gpls)[0]].table.loc[:,'ENTREZ_GENE_ID'].isna().sum()

In [ ]:
data.gpls[list(data.gpls)[0]].table.loc[:,'GB_ACC'].isna().sum()

In [ ]:
data.gpls[list(data.gpls)[0]].table.loc[:,'Gene Symbol'].isna().sum()

In [ ]:
data.gpls[list(data.gpls)[0]].table.loc[:,'Representative Public ID'].isna().sum()

In [ ]:
GEOparse.GEOTypes.GSM(data.gsms, metadata, table, columns)

In [ ]:
data.gsms[list(data.gsms)[0]].table

In [ ]:
from matplotlib import pyplot as plt
# Get expression data and metadata matrices
exprs = []
gsmNames = []
metadata = {}
for gsm_name, gsm in data.gsms.items():
    if gsm.metadata['type'][0]=='RNA':
        # Expression data
        if len(gsm.table)>0:
            tmp = gsm.table['VALUE']
            tmp.index = gsm.table['ID_REF']
            gsmNames.append(gsm_name)
            if len(exprs)==0:
                exprs = tmp.to_frame()
            else:
                exprs = pd.concat([exprs,tmp.to_frame()],axis=1)

        # Metadata
        for key,value in data.metadata.items():
            #print(key)
            #print(value)
            if (key=='characteristics_ch1' or key=='characteristics_ch2') and (len([i for i in value if i!=''])>1 or value[0].find(': ')!=-1):
                #print(value)
                tmpVal = 0
                for tmp in value:
                    splitUp = [i.strip() for i in tmp.split(':')]
                    #print(splitUp)
                    if len(splitUp)==2:
                        if not splitUp[0] in metadata:
                            metadata[splitUp[0]] = {}
                        metadata[splitUp[0]][gsm_name] = splitUp[1]
                    else:
                        if not key in metadata:
                            metadata[key] = {}
                        metadata[key][gsm_name] = splitUp[0]
            else:
                if not key in metadata:
                    metadata[key] = {}
                if len(value)==1:
                    metadata[key][gsm_name] = ' '.join([j.replace(',',' ') for j in value])

# Write expression data matrix to file
exprs.columns = gsmNames
#with open('exprs/'+gse1+'_exprs.csv','w') as outFile:
#    exprs.to_csv(outFile)

# Write metadata matrix to file
#with open('pData/'+gse1+'_pData.csv','w') as outFile:
#    outFile.write('Metadata,'+','.join(gsmNames))
#    for key in metadata:
#        tmp = [key]
#        for gsm_name in gsmNames:
#            if gsm_name in metadata[key]:
#                tmp.append(metadata[key][gsm_name])
#            else:
#                tmp.append('NA')
#        outFile.write('\n'+','.join(tmp))

# Plot boxplot of expression data
plt.boxplot(exprs.transpose(),showfliers=False)
plt.title(gse1)
pdf.savefig()
plt.close()
try:
    plt.boxplot(np.log2(exprs).transpose(),showfliers=False)
    plt.title('log2('+gse1+')')
    #pdf.savefig()
    plt.close()
except:
    pass

# Write out platform information
#for gpl_name, gpl in gse.gpls.items():
#    with open('annot/'+gse1+'_'+gpl_name+'_gpl.csv','w') as outFile:
#        gpl.table.to_csv(outFile)

In [ ]:
data.gpls[list(data.gpls.keys())[0]].table

In [ ]:
data.gpls[list(data.gpls.keys())[0]].table.loc[not data.gpls[list(data.gpls.keys())[0]].table['REFSEQ'].isna(),:]

# Testing the DB

In [ ]:
for i in range(3):
    sql = "INSERT INTO raw_dataset (Raw_ID, Raw_path) VALUES (%s, %s)" #SQL command structure
    val = ("", f"./missing_ID_{i}.txt") # Values corresponding to the rows of an instance
    cursor.execute(sql, val) # Execute the command
connection.commit()

In [ ]:
table = 'raw_dataset'
columns = tuple(['Raw_ID', 'Raw_path'])
positions = len(columns)
sql = f"INSERT INTO {table} ({','.join([str(s) for s in list(columns)])}) VALUES ({str('%s, '*len(columns)).strip().strip(',')})" #SQL command structure
val = [("", "./missing_ID_2.txt"),
      ("", "./missing_ID_3.txt"),
      ("", "./missing_ID_4.txt"),
      ("", "./missing_ID_5.txt")]# Values corresponding to the rows of an instance
cursor.executemany(sql, val) # Execute multiple lines 

connection.commit() # Commit the change to happen

print(cursor.rowcount, "record inserted.")
print(f'Last ID: {cursor.lastrowid}')

In [ ]:
table = 'preprocessed_data'
sql = f"DELETE FROM {table}" #SQL command structure
cursor.execute(sql)
connection.commit()
if str(input(f'Restore the Auto-increment ID of the table {table}? [Y/N]')).lower() in ['y','yes','ye']:
    sql = f"ALTER TABLE {table} AUTO_INCREMENT = 1" #SQL command structure
    cursor.execute(sql)
    connection.commit()

In [ ]:
table = 'raw_dataset'
columns = tuple(['Raw_ID', 'Raw_path'])
positions = len(columns)
sql = f"INSERT INTO {table} ({','.join([str(s) for s in list(columns)])}) VALUES ({str('%s, '*len(columns)).strip().strip(',')})" #SQL command structure
val = [("", "./missing_ID_2.txt"),
      ("", "./missing_ID_3.txt"),
      ("", "./missing_ID_4.txt"),
      ("", "./missing_ID_5.txt")]# Values corresponding to the rows of an instance
cursor.executemany(sql, val) # Execute multiple lines 

connection.commit() # Commit the change to happen

print(cursor.rowcount, "record inserted.")
print(f'Last ID: {cursor.lastrowid}')

In [ ]:
table = 'datasets'
sql_request = f"SELECT * FROM {table}"
cursor.execute(sql_request)

In [ ]:
insert_data = pd.DataFrame({'GEO_ID':data.name,
                           'Platform_ID':data.metadata['platform_id'][0],
                          'Expererimental_protocol':data.metadata['type'][0],
                          'Description': data.metadata['summary'][0]}, index=[0])
table = 'datasets'

#data.to_sql(table, con=engine, if_exists='append', index=False)